In [1]:
#蝦皮庫存表整理
import xlrd
import xlwt
import re
import time
import requests
from bs4 import BeautifulSoup
import tkinter as tk
import tkinter.filedialog as filedialog
import os
#時間
def run():
    today_str = time.strftime("%Y",time.localtime())+"/"+str(int(time.strftime("%m",time.localtime())))+"/"+str(int(time.strftime("%d",time.localtime())))


    #ISO 資料
    url =  "https://docs.google.com/spreadsheets/d/19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI/gviz/tq?tqx=out:html&tq=select%20B,C,D&gid=0"
    gotrq = requests.get(url)
    soup = BeautifulSoup(gotrq.text, "html.parser")
    tr = soup.select("tr")
    global isoAry
    isoAry = [] #獲取網站上的ISO 資料
    for r in tr:    
        try:
            isoAry.append([r.td.text,r.td.next_sibling.text,r.td.next_sibling.next_sibling.text])
        except:
            print('notd')

    #讀取XLS
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(filepath)
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数


    #商品資訊 陣列
    orderObj = []
    for i in range(1,rows_count): #迴圈
        orderObj.append({'note':"",'orderdate':"",'orderID':"",'name':"",'tel':"",'account':"",'shipPrice':"",'ship':"",'shipID':"",'orderPrice':"",'discount':"",'credit':"",'prds':[]})#新增新的物件
        orderObj[i-1]['orderID'] = table1.cell_value(i,0) 
        orderObj[i-1]['orderdate'] = table1.cell_value(i,5) 
        orderObj[i-1]['name'] = table1.cell_value(i,20) 
        orderObj[i-1]['tel'] = table1.cell_value(i,21) 
        orderObj[i-1]['account'] = table1.cell_value(i,3) 
        orderObj[i-1]['shipPrice'] = table1.cell_value(i,7) 
        orderObj[i-1]['ship'] = table1.cell_value(i,22) 
        orderObj[i-1]['shipID'] = table1.cell_value(i,28) 
        orderObj[i-1]['orderPrice'] = table1.cell_value(i,8) 
        orderObj[i-1]['discount'] = table1.cell_value(i,11) 
        orderObj[i-1]['note'] = table1.cell_value(i,31)+" , "+table1.cell_value(i,32)
        if table1.cell_value(i,25) == "信用卡":
            orderObj[i-1]['credit'] = table1.cell_value(i,25) 
        text = table1.cell_value(i,12) 
        if text != '商品資訊':
            text = text.replace("	","") #去除tab 大空格
            for ii in re.split("\[[0-9][0-9]*\]",text): #分割"[數字]"資料
                if ii: #如果ii有資料則輸出
                    orgtext = ii.replace("\n","").strip()
                    orderObj[i-1]['prds'].append({'prdText':orgtext,'count':prdinfo("數量:",orgtext),'price':prdinfo("價格: $",orgtext),'iso':prdinfo('商品選項貨號:',orgtext)+prdinfo('主商品貨號:',orgtext)}) #放入prd陣列中，-1是因為i是從第二列開始
            #折扣 新增一列折扣碼
            if float(orderObj[i-1]['discount']):
                orderObj[i-1]['prds'].append({'prdText':'折扣碼','count':"1",'price': float(orderObj[i-1]['discount'])*-1,'iso':"折扣碼"}) 


    #排序訂單
    orderObj.sort(key=order_time,reverse=True)
    orderObj.sort(key=order_ship)


    #寫XLS
    nwb = xlwt.Workbook()# 新增EXCEL
    nsheet = nwb.add_sheet('google試算表格式')#新SHEET
    nsheet.write(0,0,'日期')
    nsheet.write(0,1,'訂單編號')
    nsheet.write(0,2,'姓名')
    nsheet.write(0,3,'電話')
    nsheet.write(0,4,'帳號')
    nsheet.write(0,5,'國際條碼')
    nsheet.write(0,6,'產品名稱')
    nsheet.write(0,7,'產品款式')
    nsheet.write(0,8,'數量')
    nsheet.write(0,9,'價格')
    nsheet.write(0,10,'總價')
    nsheet.write(0,11,'貨運')
    nsheet.write(0,12,'運費')
    nsheet.write(0,13,'發票')
    nsheet.write(0,16,'信用卡')
    nsheet.write(0,21,'折扣')
    nsheet.write(0,22,'備註')
    write_row = 1

    for i in orderObj:
        orderstart = 0
        for ii in i['prds']:
            orderstart += 1
            if orderstart == 1:
                nsheet.write(write_row,12,int(float(i['shipPrice'])))
                nsheet.write(write_row,11,i['ship'])
                nsheet.write(write_row,22,i['note'])
                nsheet.write(write_row,13,int(float(i['orderPrice'])-float(i['discount'])))#扣掉折扣 這是訂單上的折扣(正)，非商品資訊內折扣(負號)
                if i['credit'] == '信用卡':
                    nsheet.write(write_row,16,i['credit'])
                if float(i['discount']):
                    nsheet.write(write_row,21,int(float(i['discount'])))
            nsheet.write(write_row,0,today_str)
            nsheet.write(write_row,1,i['orderID'])
            nsheet.write(write_row,2,i['name']+i['shipID'][-3:])
            nsheet.write(write_row,3,i['tel'])
            nsheet.write(write_row,4,i['account'])
            
            #商品資料
            iso_data = isodetect(ii['iso'])
            if iso_data:
                nsheet.write(write_row,5,iso_data[0]) #ISO
                nsheet.write(write_row,6,iso_data[1]) #產品名
                nsheet.write(write_row,7,iso_data[2]) #款式
            else:
                nsheet.write(write_row,6,ii['prdText'][6:ii['prdText'].find(" 價格:")])

            ct = ii['count']
            pe = ii['price']
            nsheet.write(write_row,8,ct)
            nsheet.write(write_row,9,int(float(pe)))
            nsheet.write(write_row,10,int(float(ct)*float(pe)))

            write_row += 1

    nwb.save('result.xls')

    


SyntaxError: invalid syntax (<ipython-input-1-b66341ba0ce8>, line 111)

In [63]:


def order_time(ele): #排序的KEY
    return ele['orderdate']
def order_ship(ele):
    return ele['ship']

def prdinfo(findtitle,text): #分析商品資訊
    prdtext = text.find(findtitle)#找到數量的位置
    firstsplit = prdtext+len(findtitle)+text[prdtext+len(findtitle):len(text)].find(";") #找到數量之後第一個分號位置
    
    if prdtext == -1:
        return ""
    return text[prdtext+len(findtitle):firstsplit].strip().replace(",","")#需要的片段
def isodetect(iso):
    a = 0
    for i in isoAry:
        a += 1
        if i[0].find(iso) != -1:
            if i[0] == "": #如果找到的ISO碼是空的 則返回FALSE
                return False
            return i
        
    return False
def browser():
    fname = filedialog.askopenfilename(initialdir= os.getcwd(),filetypes = (("Excel","*.xls"),("all files","*.*")))
    global filepath
    filepath =  fname  # 返回文件全路径
    pathinput.delete('0',tk.END)
    pathinput.insert(0,filepath)
    
gui=tk.Tk()
gui.title("蝦皮出貨單")
gui.geometry("450x50")
pathinput = tk.Entry(width=50)
pathinput.pack(side="left")
btn2 = tk.Button(gui, text ="瀏覽", command = browser)
btn2.pack(side="left")
btn = tk.Button(gui, text ="執行", command = run)
btn.pack(side="left")
gui.mainloop()


In [44]:
text2 = "[1]a[1]a[1]a[21]a[1]a[9]a[10]"
re.split("\[[0-9][0-9]*\]",text2) #分割"[數字]"資料

['', 'a', 'a', 'a', 'a', 'a', 'a', '']

In [58]:
ab = "abcdefgh"
ab[3:]

'defgh'